In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold, train_test_split

Pisahkan keseluruhan data menjadi data pelatihan sebanyak 75% dan data tes 
sebanyak 25%. Baik untuk data pelatihan dan data tes harus dibagi secara proporsional 
untuk masing-masing kelas label menggunakan *stratified sampling*. Tunjukkan statistik 
dari hasil pemisahan data tersebut.

In [17]:
train_df = pd.read_csv('app_train_cleaned_encoded.csv')
test_df = pd.read_csv('app_test_cleaned_encoded.csv')

print(train_df['TARGET'])
print(test_df.columns)

0        1.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
48739    0.0
48740    0.0
48741    0.0
48742    0.0
48743    0.0
Name: TARGET, Length: 48744, dtype: float64
Index(['SK_ID_CURR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY',
       'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY',
       'AMT_GOODS_PRICE', 'REGION_POPULATION_RELATIVE',
       ...
       'HOUSETYPE_MODE_block of flats', 'HOUSETYPE_MODE_specific housing',
       'HOUSETYPE_MODE_terraced house', 'WALLSMATERIAL_MODE_Block',
       'WALLSMATERIAL_MODE_Mixed', 'WALLSMATERIAL_MODE_Monolithic',
       'WALLSMATERIAL_MODE_Others', 'WALLSMATERIAL_MODE_Panel',
       'WALLSMATERIAL_MODE_Stone, brick', 'WALLSMATERIAL_MODE_Wooden'],
      dtype='object', length=236)


In [18]:

# Pisahkan fitur dan label
X = train_df.drop(columns=['SK_ID_CURR', 'TARGET'])
y = train_df['TARGET']

# Split data menjadi train dan test 75%-25% dengan stratified sampling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.25, 
    stratify=y, 
    random_state=42
)

# Cek statistik hasil split
print("Jumlah data sebelum split:")
print(y.value_counts(normalize=True))
print()

print("Jumlah data setelah split:")
print("Train set:")
print(y_train.value_counts(normalize=True))
print()
print("Test set:")
print(y_test.value_counts(normalize=True))

Jumlah data sebelum split:
TARGET
0.0    0.919436
1.0    0.080564
Name: proportion, dtype: float64

Jumlah data setelah split:
Train set:
TARGET
0.0    0.919443
1.0    0.080557
Name: proportion, dtype: float64

Test set:
TARGET
0.0    0.919416
1.0    0.080584
Name: proportion, dtype: float64


Guna memperoleh model klasifikasi terbaik untuk beberapa metode klasifikai yang harus 
diimplementasikan, gunakan keseluruhan data pelatihan menggunakan *10-fold cross 
validation*. Gunakan metode stratified sampling dalam menentukan pembagian data 
dalam 10-fold cross validation. 

In [19]:
# Set up 10-fold stratified cross-validation
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Kfold splitter
strat_kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=130)

In [20]:
train_set_kfold = []
test_set_kfold = []

for fold, (train_index, test_index) in enumerate(kfold.split(X,y)):
    print(f"Fold {fold}")
    print(f"  Train: index={train_index[:5]}... ({len(train_index)} rows)")
    print(f"  Test:  index={test_index[:5]}... ({len(test_index)} rows)")
    train_set_kfold.append(train_index)
    test_set_kfold.append(test_index)


Fold 0
  Train: index=[0 2 3 4 6]... (43869 rows)
  Test:  index=[ 1  5 24 36 54]... (4875 rows)
Fold 1
  Train: index=[0 1 2 3 4]... (43869 rows)
  Test:  index=[10 13 38 46 52]... (4875 rows)
Fold 2
  Train: index=[0 1 2 3 4]... (43869 rows)
  Test:  index=[17 21 34 37 39]... (4875 rows)
Fold 3
  Train: index=[1 3 4 5 6]... (43869 rows)
  Test:  index=[ 0  2  8 31 50]... (4875 rows)
Fold 4
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[ 9 11 14 19 33]... (4874 rows)
Fold 5
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[20 30 48 70 76]... (4874 rows)
Fold 6
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[18 22 29 32 40]... (4874 rows)
Fold 7
  Train: index=[0 1 2 3 5]... (43870 rows)
  Test:  index=[ 4  6 15 16 25]... (4874 rows)
Fold 8
  Train: index=[0 1 2 4 5]... (43870 rows)
  Test:  index=[ 3 23 45 67 71]... (4874 rows)
Fold 9
  Train: index=[0 1 2 3 4]... (43870 rows)
  Test:  index=[ 7 12 27 35 64]... (4874 rows)


NAIVE BAYES

Eksperimen untuk memperoleh model klasifikasi berbasis Naïve bayes terbaik menggunakan 10 kombinasi data pelatihan dan data validasi dari 10-fold cross validation 

Hitung akurasi, presisi, recall, f1-score, dan AUC ROC dari setiap data validasi dari langkah ke-1.  

In [21]:
from sklearn.model_selection import StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import pandas as pd

# Set up 10-fold cross-validation
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
nb_model = GaussianNB()

# Untuk menyimpan metrik dari setiap fold
results = {
    'fold': [],
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1_score': [],
    'auc_roc': []
}

# 10 kali training dan evaluasi
for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), 1):
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    # Train model
    nb_model.fit(X_tr, y_tr)
    
    # Predict
    y_val_pred = nb_model.predict(X_val)
    y_val_proba = nb_model.predict_proba(X_val)[:, 1]
    
    # Hitung metrik
    acc = accuracy_score(y_val, y_val_pred)
    prec = precision_score(y_val, y_val_pred)
    rec = recall_score(y_val, y_val_pred)
    f1 = f1_score(y_val, y_val_pred)
    auc = roc_auc_score(y_val, y_val_proba)
    
    # Simpan hasil
    results['fold'].append(fold)
    results['accuracy'].append(acc)
    results['precision'].append(prec)
    results['recall'].append(rec)
    results['f1_score'].append(f1)
    results['auc_roc'].append(auc)

# Ubah ke DataFrame untuk analisis
results_df = pd.DataFrame(results)
print(results_df)


   fold  accuracy  precision    recall  f1_score   auc_roc
0     1  0.121171   0.081205  0.962585  0.149775  0.501260
1     2  0.111324   0.081089  0.972789  0.149699  0.507753
2     3  0.124726   0.079328  0.931973  0.146211  0.486520
3     4  0.110777   0.081066  0.969492  0.149621  0.505535
4     5  0.103665   0.080944  0.976271  0.149494  0.498342
5     6  0.114059   0.081104  0.966102  0.149646  0.501010
6     7  0.111597   0.080182  0.955932  0.147954  0.500238
7     8  0.107495   0.081028  0.972881  0.149596  0.503469
8     9  0.110534   0.080567  0.965986  0.148730  0.506470
9    10  0.112175   0.080943  0.969388  0.149410  0.508930


Kemudian jalankan model yang dihasilkan pada lengkah ke-3 pada keseluruhan data pelatihan (fit the model) dan hitung kinerja (akurasi, presisi, recall, f1-score, dan AUC ROC) dari model menggunakan Naïve Bayes 

In [22]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# Definisikan StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Ambil data training dan validasi dari fold ke-0
train_idx, val_idx = list(cv.split(X_train, y_train))[0]
X_fold_train, X_fold_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
y_fold_train, y_fold_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

# Inisialisasi dan fit model
nb_model = GaussianNB()
nb_model.fit(X_fold_train, y_fold_train)

# Prediksi
y_pred = nb_model.predict(X_fold_val)
y_proba = nb_model.predict_proba(X_fold_val)[:, 1]

# Evaluasi metrik
acc = accuracy_score(y_fold_val, y_pred)
prec = precision_score(y_fold_val, y_pred, zero_division=0)
rec = recall_score(y_fold_val, y_pred, zero_division=0)
f1 = f1_score(y_fold_val, y_pred, zero_division=0)
auc = roc_auc_score(y_fold_val, y_proba)

# Tampilkan hasil
print("=== Evaluasi Model Naive Bayes (Fold ke-0) ===")
print(f"Accuracy  : {acc:.6f}")
print(f"Precision : {prec:.6f}")
print(f"Recall    : {rec:.6f}")
print(f"F1 Score  : {f1:.6f}")
print(f"AUC-ROC   : {auc:.6f}")
print("\nClassification Report:")
print(classification_report(y_fold_val, y_pred, zero_division=0))
print("Confusion Matrix:")
print(confusion_matrix(y_fold_val, y_pred))


=== Evaluasi Model Naive Bayes (Fold ke-0) ===
Accuracy  : 0.121171
Precision : 0.081205
Recall    : 0.962585
F1 Score  : 0.149775
AUC-ROC   : 0.501260

Classification Report:
              precision    recall  f1-score   support

         0.0       0.94      0.05      0.09      3362
         1.0       0.08      0.96      0.15       294

    accuracy                           0.12      3656
   macro avg       0.51      0.51      0.12      3656
weighted avg       0.87      0.12      0.10      3656

Confusion Matrix:
[[ 160 3202]
 [  11  283]]


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Misalnya X_train, y_train adalah data training yang sudah dipersiapkan
model_naive_bayes = GaussianNB()

# Melatih model dengan data training
model_naive_bayes.fit(X_train, y_train)

# Setelah model dilatih, gunakan model ini untuk prediksi
y_test_pred = model_naive_bayes.predict(X_test) 
y_test_proba = model_naive_bayes.predict_proba(X_test)[:, 1]


gunakan model Naives bayes terbaik untuk memprediksi (predict the model) semua record dari data tes. Hitung akurasi, presisi, recall, f1-score, dan AUC ROC dari model terbaik pada data tes (data yang sama sekali tidak digunakan dalam membangun model terbaik).  

In [25]:
# Prediksi pada data test
y_test_pred = model_naive_bayes.predict(X_test)  
y_test_proba = model_naive_bayes.predict_proba(X_test)[:, 1]

# Menghitung metrik evaluasi
accuracy = accuracy_score(y_test, y_test_pred)
precision = precision_score(y_test, y_test_pred, zero_division=0)
recall = recall_score(y_test, y_test_pred, zero_division=0)
f1 = f1_score(y_test, y_test_pred, zero_division=0)
auc_roc = roc_auc_score(y_test, y_test_proba)

# Menampilkan hasil
print(f"=== Evaluasi Model Naive Bayes pada Data Test ===")
print(f"Accuracy  : {accuracy:.6f}")
print(f"Precision : {precision:.6f}")
print(f"Recall    : {recall:.6f}")
print(f"F1 Score  : {f1:.6f}")
print(f"AUC-ROC   : {auc_roc:.6f}")

# Menampilkan classification report
print("\nClassification Report:")
print(classification_report(y_test, y_test_pred))

# Menampilkan confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))

=== Evaluasi Model Naive Bayes pada Data Test ===
Accuracy  : 0.109388
Precision : 0.080849
Recall    : 0.969450
F1 Score  : 0.149251
AUC-ROC   : 0.503443

Classification Report:
              precision    recall  f1-score   support

         0.0       0.93      0.03      0.07     11204
         1.0       0.08      0.97      0.15       982

    accuracy                           0.11     12186
   macro avg       0.50      0.50      0.11     12186
weighted avg       0.86      0.11      0.07     12186


Confusion Matrix:
[[  381 10823]
 [   30   952]]


HASIL EKSPERIMAN NAIVE BAYES 

Model Naive Bayes yang dilatih pada fold ke-0 menunjukkan performa yang sangat buruk, dengan akurasi rendah (~11%), precision rendah (~8%), dan AUC-ROC mendekati 0.5, yang menandakan model tidak mampu membedakan antara kelas. Meskipun recall tinggi (>96%), hal ini terjadi karena model cenderung memprediksi hampir semua data sebagai kelas positif, akibat dari ketidakseimbangan kelas yang signifikan. Secara keseluruhan, model ini tidak cocok digunakan pada dataset ini